## Load Modules & setup

In [1]:
!pip install catboost -q

Mounted at /content/drive
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [3]:
#import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## Load data

In [4]:
path = ''
pixel_df = pd.read_csv(path + 'pixel_df.csv')
tile_df = pd.read_csv(path + 'tile_df.csv')
tile_field_df = pd.read_csv(path + 'tile_field_df.csv')
field_crop_pair = pd.read_csv(path + 'field_crop_df.csv')

display(pixel_df.head(), tile_df.head(), tile_field_df.head(), field_crop_pair.head())

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,field_id,x,y
0,43,39,38,38,41,54,63,61,64,12,57,37,757,23,43
1,43,39,38,38,42,57,67,63,72,12,63,42,757,23,44
2,43,39,38,37,41,59,69,65,78,12,68,43,757,24,44
3,43,38,37,36,41,59,69,64,78,12,68,43,757,25,44
4,43,39,38,38,42,57,67,64,72,12,63,42,757,23,45


,tile_id,minx,miny,maxx,maxy
0,28852,82.293829,27.327138,82.319974,27.350486
1,d987c,81.542781,27.193544,81.568744,27.216758
2,ca1d4,81.543229,27.285989,81.569214,27.309203
3,2ec18,87.712321,25.343604,87.737900,25.366846
4,7575d,83.676445,19.104290,83.701132,19.127756


,field_id,tile_id
0,757,28852
1,756,28852
2,1372,28852
3,1374,28852
4,1986,d987c


,field_id,crop_id
0,757,6
1,756,6
2,1372,5
3,1374,1
4,1986,4


## Preprocessing & Feature Engineering

In [5]:
# real x, y coordinates from the tile_df xmin, ymin, xmax, ymax

## first merge the tile_df and tile_field_df
field_info = tile_field_df.merge(tile_df, on='tile_id', how='left')

## from the field info calculate the real x, y coordinates
for field_id in tqdm(field_info.field_id.unique()):
    maxx, maxy, minx, miny = field_info[field_info.field_id == field_id][['maxx', 'maxy', 'minx', 'miny']].values[0]
    pixel_df.loc[pixel_df.field_id == field_id, 'x'] = pixel_df.loc[pixel_df.field_id == field_id, 'x'] * (maxx - minx) + minx
    pixel_df.loc[pixel_df.field_id == field_id, 'y'] = pixel_df.loc[pixel_df.field_id == field_id, 'y'] * (maxy - miny) + miny

all_data = pixel_df.copy()

100%|██████████| 7081/7081 [00:45<00:00, 156.97it/s]


In [6]:
coco = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06',  'B08','B08',  'B09','B11', 'B12']
for coc in coco:
  all_data['norm_'+coc] = all_data[coc] / all_data[coco].mean(axis=1)

In [8]:
from itertools import combinations

#calculate  normalized difference between all bands combinations
band_cols = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09','B11', 'B12']
for col1,col2 in list(combinations(band_cols, 2)):
  all_data['ND'+col1+'_'+col2]=(all_data[col1]-all_data[col2])/(all_data[col1]+all_data[col2])

In [9]:
all_data['datt1'] = (all_data.B08-all_data.B05)/(all_data.B08-all_data.B04)
all_data['datt2'] = all_data.B04/(all_data.B03*all_data.B05)
all_data['datt2'] = all_data.B8A/(all_data.B03*all_data.B05)
all_data['DVI'] = 2.4*all_data.B08 - all_data.B04
all_data['EVI'] = (all_data.B08 - all_data.B04) / (all_data.B08 + 6 * all_data.B04 - 7.5 * all_data.B02 + 1)

In [10]:
all_data['AFRI_1'] = (all_data['B8A']-0.66*all_data['B11'])/(all_data['B8A']+0.66*all_data['B11'])
all_data['AFRI_2'] = (all_data['B8A']-0.5*all_data['B12'])/(all_data['B8A']+0.5*all_data['B12'])
all_data['ARI'] = (1/all_data['B03']-(1/all_data['B05']))  
all_data['ARVI_0.5'] = (all_data['B08']-(all_data['B04']-0.5*(all_data['B02']-all_data['B04'])))/(all_data['B08']+(all_data['B04']-0.5*(all_data['B02']-all_data['B04'])))
all_data['ARVI_2'] = -0.18+1.17*((all_data['B08']-all_data['B04'])/(all_data['B08']+all_data['B04']))
all_data['ATSAVI'] = (1.22*(all_data['B08']-1.22*all_data['B04']-0.03))/(all_data['B08']+all_data['B04']-1.22*0.03+0.08*(1+1.22**2))
all_data['AVI'] = 2*all_data['B8A']-all_data['B04']
all_data['BRI'] = all_data['ARI']/all_data['B06']
all_data['CCCI'] = all_data['NDB05_B08']/all_data['NDB04_B08']
all_data['CRI550'] = (1/all_data['B02']-(1/all_data['B03']))
all_data['CRI700'] = (1/all_data['B02']-(1/all_data['B05']))
all_data['CVI'] = all_data['B08']*all_data['B04']/(all_data['B03']**2)

In [11]:
%%capture
all_data['NGRDI'] = (all_data['B03'] - all_data['B04']) / (all_data['B03'] + all_data['B04'])
all_data['TGI'] = (190 * (all_data['B04'] - all_data['B03']) - 120 * (all_data['B04'] - all_data['B02']))
all_data['NDVI'] = (all_data['B08'] - all_data['B04']) / (all_data['B08'] + all_data['B04'])
all_data['GNDVI'] = (all_data['B08'] - all_data['B03']) / (all_data['B08'] + all_data['B03'])
all_data['TCARI/OSAVI'] = 3 * ((all_data['B05'] - all_data['B04']) - 0.2 * (all_data['B05'] - all_data['B03']) * (all_data['B05'] / all_data['B04'])) / ((1 + 0.16) * (all_data['B07'] - all_data['B04']) * (all_data['B07'] + all_data['B04'] + 0.16))
all_data['MTCI'] = (all_data['B06'] - all_data['B05']) / (all_data['B05'] - all_data['B04']+0.1)
all_data['CVI'] = (all_data['B08'] / all_data['B03']) / (all_data['B03'] / all_data['B04'])
all_data['CI_red-edge'] = (all_data['B08'] / all_data['B05']) 
all_data['IRECI'] = (all_data['B07']-all_data['B04']) / (all_data['B05'] / all_data['B06'])
all_data['SR'] =all_data['B05']/all_data['B04']
all_data['NDWI'] = (all_data['B03'] - all_data['B05']) / (all_data['B03'] + all_data['B05'])
all_data['NDII'] = (all_data['B05']*0.819 - all_data['B09']*16.49) / (all_data['B05']*0.819 + all_data['B09']*16.49)
all_data['MNDWI'] = (all_data['B03'] - all_data['B09']*1.65) / (all_data['B03'] + all_data['B09']*1.65)
all_data['RVI'] = all_data['B08']/all_data['B04']

In [12]:
band_cols = all_data.columns.difference(['crop_id','field_id','source','x','y'])

def add_prefix(base,cols=band_cols):
  return [base+'_'+col for col in cols]

def lower(x):
  return np.percentile(x, 25)

def upper(x):
  return np.percentile(x, 75)

In [13]:
grouped_data = all_data.groupby(['field_id']).mean().reset_index()
grouped_data['size'] = all_data.groupby(['field_id']).size().values
grouped_data[add_prefix('median')] = all_data.groupby(['field_id'])[band_cols].median().values
grouped_data[add_prefix('max')] = all_data.groupby(['field_id'])[band_cols].max().values
grouped_data[add_prefix('quantile25')] = all_data.groupby(['field_id'])[band_cols].quantile(0.25).values
grouped_data[add_prefix('quantile75')] = all_data.groupby(['field_id'])[band_cols].quantile(0.75).values
grouped_data[add_prefix('lower')] = all_data.groupby(['field_id'])[band_cols].agg(lower).values
grouped_data[add_prefix('upper')] = all_data.groupby(['field_id'])[band_cols].agg(upper).values

In [14]:
coordinates=['x','y']

grouped_data[add_prefix('median',coordinates)] = all_data.groupby(['field_id'])[coordinates].median().values
grouped_data[add_prefix('mean',coordinates)] = all_data.groupby(['field_id'])[coordinates].mean().values
grouped_data[add_prefix('max',coordinates)] = all_data.groupby(['field_id'])[coordinates].max().values
grouped_data[add_prefix('min',coordinates)] = all_data.groupby(['field_id'])[coordinates].min().values
grouped_data[add_prefix('ptp',coordinates)] = grouped_data[add_prefix('max',coordinates)].values-grouped_data[add_prefix('min',coordinates)].values
grouped_data[add_prefix('unique',coordinates)] = all_data.groupby(['field_id'])[coordinates].nunique().values

In [15]:
# merge pixel dataframe to field_crop_pair dataframe
grouped_data['field_id'] = grouped_data['field_id'].apply (lambda x:str(int(x)))
field_crop_pair['field_id'] = field_crop_pair['field_id'].apply (lambda x:str(int(x)))
grouped_data = pd.merge(grouped_data, field_crop_pair , on='field_id',how='left' )

In [16]:
grouped_data.fillna(-9999,inplace=True)
grouped_data.replace([np.inf], 9999, inplace=True)
train_df = grouped_data[grouped_data['field_id'].isin(field_crop_pair['field_id'].values)].reset_index(drop=True)
test_df = grouped_data[~grouped_data['field_id'].isin(field_crop_pair['field_id'].values)].reset_index(drop=True)

## Model building and Experiment

In [17]:
train_cols = grouped_data.columns.difference(['source','crop_id','field_id'])

In [18]:
features = train_cols
num_feature = features
drop_columns = []
corr = grouped_data[num_feature].corr()
# Drop highly correlated features 
columns = np.full((corr.shape[0],), True, dtype=bool)
max_corr = 0.995
for i in tqdm(range(corr.shape[0])):
    for j in range(i+1, corr.shape[0]):
        if abs(corr.iloc[i,j]) >= max_corr :
            if columns[j]:
                columns[j] = False
drop_columns = grouped_data[num_feature].columns[columns == False].values
print('drop_columns',len(drop_columns),drop_columns)

100%|██████████| 834/834 [00:08<00:00, 95.31it/s] 

drop_columns 416 ['ATSAVI' 'NDB03_B08' 'NDB03_B09' 'NDB04_B08' 'NDB04_B8A' 'NDB07_B11'
 'NDB07_B12' 'NDB08_B09' 'NDB08_B11' 'NDB08_B12' 'NDB8A_B09' 'NDB8A_B11'
 'NDB8A_B12' 'NDVI' 'NDWI' 'NGRDI' 'SR' 'lower_ATSAVI' 'lower_B01'
 'lower_CRI700' 'lower_MNDWI' 'lower_NDB01_B09' 'lower_NDB02_B05'
 'lower_NDB02_B09' 'lower_NDB02_B11' 'lower_NDB03_B09' 'lower_NDB06_B09'
 'lower_NDB07_B09' 'lower_NDB07_B11' 'lower_NDB07_B12' 'lower_NDB08_B09'
 'lower_NDB08_B11' 'lower_NDB08_B12' 'lower_NDB8A_B09' 'lower_NDB8A_B11'
 'lower_NDB8A_B12' 'lower_NDVI' 'lower_NDWI' 'lower_NGRDI' 'max_ATSAVI'
 'max_NDB03_B09' 'max_NDB07_B11' 'max_NDB07_B12' 'max_NDB8A_B11'
 'max_NDB8A_B12' 'max_NDVI' 'max_NDWI' 'max_NGRDI' 'max_datt1'
 'median_AFRI_1' 'median_AFRI_2' 'median_ARI' 'median_ARVI_0.5'
 'median_ARVI_2' 'median_ATSAVI' 'median_B01' 'median_B02' 'median_B03'
 'median_B04' 'median_B05' 'median_B09' 'median_B11' 'median_B12'
 'median_BRI' 'median_CI_red-edge' 'median_CRI700' 'median_CVI'
 'median_GNDVI' 'media

In [19]:
train_cols = list(train_cols.difference(drop_columns))

In [20]:
X = train_df[train_cols]
y = train_df["crop_id"]
test_data = test_df[train_cols]

In [21]:
from imblearn.under_sampling import RandomUnderSampler

def get_X_y(seed=0):
  global X, y
  strategy = {
      1.0: 900, 4.0: 800, 2.0: 500, 9.0: 293, 6.0: 163,
      36.0: 129, 3.0: 103, 13.0: 59, 8.0: 48, 15.0: 41,
      5.0: 23, 16.0: 16, 14.0: 14
      }
  sampler = RandomUnderSampler(sampling_strategy=strategy, random_state=seed)
  return sampler.fit_resample(X, y)

In [22]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold


def get_model1(seed=0):
  params = {
      'iterations':900,
      'random_state':seed,
      'bootstrap_type':'Bernoulli',
      'max_depth': 7,
      'learning_rate': 0.008436034013684798,
      'l2_leaf_reg': 0.8908564261285311,
      'auto_class_weights':'Balanced',
      'task_type':'GPU',
      'verbose' : 200,
  } 
  return CatBoostClassifier(**params)


def get_model2(seed=0):
  params = {
      'learning_rate': 0.011655679354117265,
      'subsample': 0.9586053451656916,
      'colsample_bytree': 0.9062805522038428,
      'max_depth': 8,
      'objective' : 'multiclass',
      'n_estimators' :500,
      'random_state':seed,
      # 'device' : 'gpu',
      'verbose' : 200,
      'class_weight' : 'balanced'
      }

  return LGBMClassifier(**params)


def get_model3(seed=0):
  params = {
      'learning_rate': 0.014378176510840895,
      'max_depth': 7,
      'iterations': 601,
      'random_state': seed,
      'bootstrap_type': 'Bernoulli',
      'auto_class_weights': 'Balanced',
      'task_type': 'GPU',
      'l2_leaf_reg': 0.8908564261285311
      }
  return CatBoostClassifier(**params)

#weighted multi-class log loss
def weighted_log_loss(y_true, y_pred):
    weights = np.ones(13, dtype='float32')
    class_proportions = np.array([
       1.        , 0.48744461, 0.05071393, 0.80797637, 0.01132447,
       0.08025603, 0.02363368, 0.14426391, 0.02904973, 0.00689316,
       0.0201871 , 0.00787789, 0.06351551
    ])
    y_true = pd.get_dummies(y_true).values
    y_true_weighted = (y_true * weights ) / class_proportions

    loss_num = (y_true_weighted * np.log(y_pred))
    loss = -1*np.sum(loss_num) / np.sum(weights)
    
    return loss


def eval_model(model, ns=5):
    Xs, ys = get_X_y()
    fold_loss = []
    skf = StratifiedKFold(n_splits=ns, shuffle=True, random_state=21)
    print('evaluating... fold ', end='')
    for fold_, (tr, vl) in enumerate(skf.split(Xs, ys)):
        print(fold_, end=" ")
        
        X_tr, y_tr = Xs.loc[tr], ys.loc[tr]
        X_vl, y_vl = Xs.loc[vl], ys.loc[vl]

        model.fit(X_tr, y_tr, verbose=0)
        pr = model.predict_proba(X_vl)

        loss = weighted_log_loss(y_vl, pr)
        fold_loss += [loss]
    print(" done")

    return sum(fold_loss) / len(fold_loss)


In [ ]:
# eval_model(get_model2(), ns=5)
# # 869.2071785376504

evaluating... fold 0 1 2 3 4  done


1030.3213755101622

In [ ]:
# eval_model(get_model1())
# 398.19544298521544 729.9011217672762

evaluating... fold 0 1 2 3 4  done


749.448065018252

In [23]:
predf = []
models = []
for seed_ in range(10):
  print('\n'+'-'*10+" seed =",seed_,'-'*10)

  model = get_model1(seed=seed_)
  Xs, ys = get_X_y(seed=seed_)
  model.fit(Xs, ys, verbose=500)
  predcf = model.predict_proba(test_data)
  models.append(model)
  
  model = get_model2(seed=seed_)
  model.fit(Xs, ys, verbose=500)
  predlf = model.predict_proba(test_data)
  models.append(model)

  predf.append(predcf*.7+predlf*.3)



---------- seed = 0 ----------
0:	learn: 2.5417048	total: 92.3ms	remaining: 1m 22s
500:	learn: 0.6458963	total: 23.1s	remaining: 18.4s
899:	learn: 0.4447927	total: 40.6s	remaining: 0us

---------- seed = 1 ----------
0:	learn: 2.5430284	total: 89.3ms	remaining: 1m 20s
500:	learn: 0.6569970	total: 23.1s	remaining: 18.4s
899:	learn: 0.4547130	total: 40.8s	remaining: 0us

---------- seed = 2 ----------
0:	learn: 2.5400251	total: 96.4ms	remaining: 1m 26s
500:	learn: 0.6548207	total: 23.3s	remaining: 18.5s
899:	learn: 0.4524727	total: 40.8s	remaining: 0us

---------- seed = 3 ----------
0:	learn: 2.5409372	total: 96.4ms	remaining: 1m 26s
500:	learn: 0.6553231	total: 23.2s	remaining: 18.5s
899:	learn: 0.4573985	total: 40.5s	remaining: 0us

---------- seed = 4 ----------
0:	learn: 2.5420974	total: 89.1ms	remaining: 1m 20s
500:	learn: 0.6580024	total: 23.1s	remaining: 18.4s
899:	learn: 0.4552231	total: 42.3s	remaining: 0us

---------- seed = 5 ----------
0:	learn: 2.5416499	total: 95ms	remain

In [24]:
test_pred = np.mean(predf, axis=0)

In [25]:
crop_dict = {
    1: 'Wheat', 2: 'Mustard', 3: 'Lentil',  4: 'No Crop', 6: 'Sugarcane',
    8: 'Garlic', 15: 'Potato', 5: 'Green pea', 16: 'Bersem', 14: 'Coriander',
    13: 'Gram', 9: 'Maize', 36: 'Rice'
    }
def labeler(labeled):
    crop_label = np.array([crop_dict.get(f'{int(i)}') for i in labeled])
    return crop_label

In [26]:
crop_columns = [crop_dict.get(i) for i in y.unique()]
sub  = pd.DataFrame(test_pred, columns=crop_columns)
sub['field_id'] = test_df['field_id'].values
sub=sub[['field_id'] + crop_columns]
sub.to_csv('submission.csv', index=False)
sub

,field_id,Wheat,Mustard,Lentil,No Crop,Sugarcane,Garlic,Potato,Green pea,Bersem,Coriander,Gram,Maize,Rice
0,11,0.071314,0.190105,0.073509,0.044831,0.019203,0.543993,0.006467,0.006412,0.004104,0.009107,0.015929,0.009856,0.005170
1,13,0.361187,0.208749,0.080314,0.150331,0.062026,0.057936,0.016793,0.003456,0.009799,0.007517,0.007276,0.026684,0.007932
2,19,0.150218,0.216380,0.288761,0.052437,0.020814,0.223397,0.006722,0.002894,0.005033,0.017892,0.005925,0.006812,0.002715
3,21,0.045399,0.167796,0.099655,0.214005,0.021946,0.312301,0.023934,0.005974,0.007388,0.006378,0.060701,0.027783,0.006740
4,25,0.079700,0.037504,0.121934,0.715310,0.007586,0.018573,0.004645,0.000898,0.003501,0.002186,0.001742,0.005055,0.001366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,7319,0.013329,0.018014,0.001799,0.039252,0.001872,0.004628,0.001472,0.817898,0.001135,0.001697,0.007830,0.003047,0.088027
1526,7325,0.012180,0.024211,0.001396,0.022476,0.001357,0.003317,0.001082,0.900896,0.000824,0.001345,0.011034,0.001387,0.018495
1527,7329,0.008130,0.016529,0.001376,0.015980,0.001323,0.002462,0.001034,0.785942,0.000877,0.001367,0.154858,0.001347,0.008776
1528,7330,0.005950,0.008659,0.000838,0.021666,0.000950,0.001555,0.000752,0.924089,0.000583,0.001036,0.020222,0.001142,0.012558


In [27]:
pd.Series(np.argmax(sub[sub.columns[1:]].values, axis=1)).map(dict(zip(range(13), sub.columns[1:]))).value_counts()

No Crop      295
Wheat        255
Garlic       228
Green pea    221
Mustard      181
Rice         124
Lentil        94
Potato        46
Gram          44
Bersem        31
Maize          6
Sugarcane      4
Coriander      1
dtype: int64